In [1]:
%cd /content

/content


In [ ]:
# http://boofcv.org/index.php?title=File:Calibration_A4_chessboard_9x6.png

In [ ]:
!git clone https://github.com/niconielsen32/ComputerVision.git

Cloning into 'ComputerVision'...
remote: Enumerating objects: 927, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 927 (delta 23), reused 40 (delta 10), pack-reused 856
Receiving objects: 100% (927/927), 201.95 MiB | 25.15 MiB/s, done.
Resolving deltas: 100% (273/273), done.
Updating files: 100% (375/375), done.


In [ ]:
!git clone

In [ ]:
%cp -r /content/ComputerVision/cameraCalibration /content/drive/MyDrive/comp472

In [ ]:
# Imports PIL module
from PIL import Image

# open method used to open different extension image file
im = Image.open(r"/content/drive/MyDrive/comp472/cameraCalibration/calibration/Image__2018-10-05__10-34-23.png")

# This method will show image in any image viewer
im.show()

In [ ]:
im.size

(1440, 1080)

#Create your own pattern


Now, if you want to create your own pattern, you will need python to use https://github.com/opencv/opencv/blob/4.x/doc/pattern_tools/gen_pattern.py

Example

create a checkerboard pattern in file chessboard.svg with 9 rows, 6 columns and a square size of 20mm:

In [ ]:
# https://markhedleyjones.com/projects/calibration-checkerboard-collection
# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

In [2]:
%cd /content/drive/MyDrive/comp472/cameraCalibration/calibration

/content/drive/MyDrive/comp472/cameraCalibration/calibration


In [8]:
!python gen_pattern.py -o chessboard.svg --rows 9 --columns 6 --type checkerboard --square_size 20

In [9]:
%cd /content/drive/MyDrive/comp472/cameraCalibration/calibration

/content/drive/MyDrive/comp472/cameraCalibration/calibration


In [13]:
!python cameraCalibration.py

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.10/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [18]:
import numpy as np
import cv2 as cv
import glob
from google.colab.patches import cv2_imshow


################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (24,17)
frameSize = (1440,1080)



# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 20
objp = objp * size_of_chessboard_squares_mm


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


images = glob.glob('*.png')

for image in images:

    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if ret == True:

        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv2_imshow(img)
        cv.waitKey(1000)


cv.destroyAllWindows()




############## CALIBRATION #######################################################

ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)


Output hidden; open in https://colab.research.google.com to view.

In [19]:
############## UNDISTORTION #####################################################

img = cv.imread('Image__2018-10-05__10-29-04.png')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))



# Undistort
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult1.png', dst)



# Undistort with Remapping
mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult2.png', dst)




# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.05429344718058781
